# 네이버 환전 페이지를 파싱해보자

## 파싱/크롤링을 할 때 셀레늄이 꼭 필요한가?
- 그렇지 않음
- 셀레늄은 페이지가 동적으로 그려지는(ajax 등) 경우에 필요
- 일반적으로 정적인 페이지는 셀레늄이 없어도 가능 

In [102]:
import datetime
import urllib.request as req  # 웹 요청할 때 사용할 파이썬 기본 라이브러리
from bs4 import BeautifulSoup  # 일반적으로 돔 파싱할때 사용하는 라이브러리
import numpy as np
import pandas as pd

## 파싱할 페이지의 html을 가져오기

In [22]:
url = "https://finance.naver.com/marketindex/exchangeList.nhn"

data = req.urlopen(url)
html = data.read().decode("euc-kr")  # 

## HTML 파서 지정

In [107]:
soup = BeautifulSoup(html, 'html.parser')
soup


<html lang="ko">
<head>
<title>네이버 금융</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<link href="/css/finance.css?20200729152042" rel="stylesheet" type="text/css"/>
<script language="javascript">document.domain="naver.com";</script>
<script src="/js/info/jindo.min.ns.1.5.3.euckr.js" type="text/javascript"></script>
<script src="/js/lcslog.js?20200729152042" type="text/javascript"></script>
</head>
<body>
<div class="tbl_area">
<table border="1" class="tbl_exchange" summary="환전 고시 환율 리스트">
<caption>환전 고시 환율</caption>
<colgroup>
<col width="162"/>
<col width="92"/>
<col width="92"/>
<col width="92"/>
<col width="93"/>
<col width="92"/>
<col width="90"/>
</colgroup>
<thead>
<tr>
<th class="th_ex1" rowspan="2">
<a href="#" onclick="javascript:changeOrder('exchange'); return false;"><span>통화명</span></a></th>
<th class="th_ex2" rowspan="

## 웹 데이터 긁어오기

In [120]:
thead = soup.select_one("table > thead")  # 데이터의 헤더인 것 같다
table_data = soup.select("table > tbody > tr")  # html 테이블의 실제 데이터가 담긴 곳들
now_time = datetime.datetime.now().strftime("%Y.%m.%d %H%M%S")

# 테이블 헤더를 보면 알겠지만 특정 컬림이 분리되어있어서 데이터와 헤더가 맞지 않음
# 이를 해결하기 위해 별도의 헤더를 리스트로 생성해 줬음
head = ["통화명", "매매기준율", "현찰/사실 때", "현찰/파실 때", "송금/보내실 때", "송금/받으실 때", "미화환산율"]
body = []

# 실제 테이블 내용을 순회하면서 필요한 데이터만 빼옴
for row in table_data:
    new_row = [i.text.strip() for i in row.select("td")]
    body.append(new_row)

print(now_time)
print(head)
print(body)

2020.07.30 010044
['통화명', '매매기준율', '현찰/사실 때', '현찰/파실 때', '송금/보내실 때', '송금/받으실 때', '미화환산율']
[['미국 USD', '1,194.00', '1,214.89', '1,173.11', '1,205.70', '1,182.30', '1.000'], ['유럽연합 EUR', '1,401.40', '1,429.28', '1,373.52', '1,415.41', '1,387.39', '1.174'], ['일본 JPY (100엔)', '1,137.09', '1,156.98', '1,117.20', '1,148.23', '1,125.95', '0.952'], ['중국 CNY', '170.56', '179.08', '162.04', '172.26', '168.86', '0.143'], ['홍콩 HKD', '154.06', '157.09', '151.03', '155.60', '152.52', '0.129'], ['대만 TWD', '40.78', '46.12', '37.93', '0.00', '0.00', '0.034'], ['영국 GBP', '1,548.26', '1,578.76', '1,517.76', '1,563.74', '1,532.78', '1.297'], ['오만 OMR', '3,101.22', '3,377.22', '2,915.15', '0.00', '0.00', '2.597'], ['캐나다 CAD', '893.71', '911.31', '876.11', '902.64', '884.78', '0.749'], ['스위스 CHF', '1,300.80', '1,326.42', '1,275.18', '1,313.80', '1,287.80', '1.089'], ['스웨덴 SEK', '136.23', '139.56', '132.90', '137.59', '134.87', '0.114'], ['호주 AUD', '856.46', '873.33', '839.59', '865.02', '847.90', '0.717'], 

## 좀 더 보기 좋은 형태로 가공하기
- 주피터 노트북은 pandas 데이터, matplotlib 등의 데이터를 보기 좋게 정리해줌
- 이를 이용하여 깔끔한 형태의 데이터를 만들어 보자

In [121]:
df = pd.DataFrame(data=body, columns=head)
print(now_time)
df

2020.07.30 010044


,통화명,매매기준율,현찰/사실 때,현찰/파실 때,송금/보내실 때,송금/받으실 때,미화환산율
0,미국 USD,"1,194.00","1,214.89","1,173.11","1,205.70","1,182.30",1.000
1,유럽연합 EUR,"1,401.40","1,429.28","1,373.52","1,415.41","1,387.39",1.174
2,일본 JPY (100엔),"1,137.09","1,156.98","1,117.20","1,148.23","1,125.95",0.952
3,중국 CNY,170.56,179.08,162.04,172.26,168.86,0.143
4,홍콩 HKD,154.06,157.09,151.03,155.60,152.52,0.129
5,대만 TWD,40.78,46.12,37.93,0.00,0.00,0.034
6,영국 GBP,"1,548.26","1,578.76","1,517.76","1,563.74","1,532.78",1.297
7,오만 OMR,"3,101.22","3,377.22","2,915.15",0.00,0.00,2.597
8,캐나다 CAD,893.71,911.31,876.11,902.64,884.78,0.749
9,스위스 CHF,"1,300.80","1,326.42","1,275.18","1,313.80","1,287.80",1.089


## 파일로 저장하기

In [124]:
df.to_csv(now_time+".csv", sep=",", index=False)

## 저장된 파일 불러오기

In [125]:
new_df = pd.read_csv(now_time+".csv")
new_df

,통화명,매매기준율,현찰/사실 때,현찰/파실 때,송금/보내실 때,송금/받으실 때,미화환산율
0,미국 USD,"1,194.00","1,214.89","1,173.11","1,205.70","1,182.30",1.000
1,유럽연합 EUR,"1,401.40","1,429.28","1,373.52","1,415.41","1,387.39",1.174
2,일본 JPY (100엔),"1,137.09","1,156.98","1,117.20","1,148.23","1,125.95",0.952
3,중국 CNY,170.56,179.08,162.04,172.26,168.86,0.143
4,홍콩 HKD,154.06,157.09,151.03,155.60,152.52,0.129
5,대만 TWD,40.78,46.12,37.93,0.00,0.00,0.034
6,영국 GBP,"1,548.26","1,578.76","1,517.76","1,563.74","1,532.78",1.297
7,오만 OMR,"3,101.22","3,377.22","2,915.15",0.00,0.00,2.597
8,캐나다 CAD,893.71,911.31,876.11,902.64,884.78,0.749
9,스위스 CHF,"1,300.80","1,326.42","1,275.18","1,313.80","1,287.80",1.089


## DONE